## Рецепты

In [5]:
!pip install transformers datasets huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
squad_v2 = True
model_checkpoint = "timpal0l/mdeberta-v3-base-squad2"
batch_size = 16

In [8]:
from datasets import load_dataset, load_metric

In [9]:
datasets = load_dataset("blinoff/medical_qa_ru_data")

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
datasets

DatasetDict({
    train: Dataset({
        features: ['date', 'categ', 'theme', 'desc', 'ans', 'spec10'],
        num_rows: 190335
    })
})

In [10]:
datasets["train"][0]

{'date': '8 Октября 2017, 11:55',
 'categ': 'Оториноларингология',
 'theme': 'Применение Ларипронта.',
 'desc': 'Ларипронт 20 талеток,через каждые 2-3 часа.Очень понравились,но пока принимала,чувствовала себя хорошо. Закончились-все сиптомы опять проявились. Нигде не найду как долго их можно принимать. Скажите пожалуйста.',
 'ans': 'Что вы им лечите? Длительность приема Ларипронта индивидуальна и устанавливается лечащим врачом.;\nУточните у своего лечащего врача.',
 'spec10': 'Отоларинголог'}

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
max_length = 396  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

In [14]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["ans"],
        examples["desc"],
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )


    return tokenized_examples

In [15]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

### Fine-tuning the model

In [16]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [17]:


model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)



In [18]:
from transformers import default_data_collator

data_collator = default_data_collator

In [20]:
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

TypeError: ignored